In [ ]:
import pandas as pd
from tbdynamics.tools.plotting import plot_model_vs_actual
import nevergrad as ng
from tbdynamics.settings import CM_PATH
# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.camau.calibration.utils import get_bcm, load_targets
from multiprocessing import cpu_count

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':False
}
bcm = get_bcm(params, covid_effects, homo_mixing=False)

In [ ]:
# Set the number of workers for parallel optimization
orunner = optimize_model(bcm, opt_class=ng.optimizers.TwoPointsDE, num_workers=cpu_count())
for i in range(8):
    rec = orunner.minimize(500)
mle_params = rec.value[1]
mle_params

In [ ]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets(CM_PATH / 'targets.yml')

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [ ]:
derived_df_0['detection_rate'].plot()

In [ ]:
derived_df_0['mortality'].plot()

In [ ]:
# derived_df_0['act3_trial_adults_pop'].plot()
# derived_df_0['act3_control_adults_pop'].plot()
# derived_df_0[[f'act3_{act3_stratum}_adults_pop' for act3_stratum in ["trial","control", "other"]]].plot()

In [ ]:
derived_df_0.loc[2014:2019,[f'total_populationXact3_{act3_stratum}' for act3_stratum in ["trial","control"]]].plot()

In [ ]:
derived_df_0.loc[2014:2019,'acf_detectionXact3_trialXorgan_pulmonary'].plot()

In [ ]:
derived_df_0.loc[2016:2019,'acf_detectionXact3_controlXorgan_pulmonary'].plot()